In [1]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

digits = load_digits()
X, y = digits.data, digits.target

# Select multiple classes 
mask = (y >= 0) & (y <= 9)
X, y = X[mask], y[mask]

# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA to match number of qubits
nqubits = 8  # Slightly increased for complexity
pca = PCA(n_components=nqubits)
X_reduced = pca.fit_transform(X_scaled)

# Train-test split
xs_tr, xs_test, y_tr, y_test = train_test_split(X_reduced, y, test_size=0.25, random_state=42)


In [ ]:
# Classical SVM
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

start_time = time.time()

# Classical SVM with RBF kernel (typical classical benchmark)
svc_rbf = SVC(kernel='rbf', random_state=42)
svc_rbf.fit(xs_tr, y_tr)

# Evaluate
y_pred_rbf = svc_rbf.predict(xs_test)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

print(f"Classical RBF SVM accuracy: {accuracy_rbf:.2%}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")


In [ ]:
import time
from qiskit.circuit.library import ZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from sklearn.metrics import accuracy_score

start_time = time.time()

# GPU-accelerated simulator
simulator_gpu = AerSimulator(device='GPU')

# Define ZFeatureMap (no entanglement)
# nqubits = 10  # same as your original setup
zfm = ZFeatureMap(feature_dimension=nqubits, reps=2)

# FidelityQuantumKernel setup
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel_gpu = FidelityQuantumKernel(feature_map=zfm, fidelity=fidelity)

# QSVC using FidelityQuantumKernel
qsvm_gpu = QSVC(quantum_kernel=quantum_kernel_gpu)
qsvm_gpu.fit(xs_tr, y_tr)

# Evaluate and print GPU-accelerated results
y_pred_gpu = qsvm_gpu.predict(xs_test)
accuracy_gpu = accuracy_score(y_test, y_pred_gpu)
print(f"GPU-accelerated QSVC accuracy (no entanglement): {accuracy_gpu:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


In [ ]:
import time
from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from sklearn.metrics import accuracy_score

start_time = time.time()

# GPU-accelerated simulator
simulator_gpu = AerSimulator(device='GPU')


# Define ZZFeatureMap
# nqubits = 8
zzfm = ZZFeatureMap(feature_dimension=nqubits, reps=2, entanglement='full')

# FidelityQuantumKernel setup
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel_gpu = FidelityQuantumKernel(feature_map=zzfm, fidelity=fidelity)

# QSVC using FidelityQuantumKernel
qsvm_gpu = QSVC(quantum_kernel=quantum_kernel_gpu)
qsvm_gpu.fit(xs_tr, y_tr)

# Evaluate and print GPU-accelerated results
y_pred_gpu = qsvm_gpu.predict(xs_test)
accuracy_gpu = accuracy_score(y_test, y_pred_gpu)
print(f"GPU-accelerated QSVC accuracy: {accuracy_gpu:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")



In [ ]:
import qiskit_machine_learning
print(qiskit_machine_learning.__version__)


In [ ]:
import qiskit_algorithms
print(qiskit_algorithms.__version__)


In [ ]:
import qiskit
# Note: The package is installed as 'qiskit-aer',
# but imported as 'qiskit_aer'
import qiskit_aer
import sys

print(f"Python version: {sys.version}")

try:
    print(f"Qiskit version: {qiskit.__version__}")
except AttributeError:
    print("Could not retrieve Qiskit version using __version__.")
except ImportError:
    print("Qiskit is not installed or importable.")

try:
    print(f"Qiskit Aer version: {qiskit_aer.__version__}")
except AttributeError:
     print("Could not retrieve Qiskit Aer version using __version__.")
except ImportError:
    print("Qiskit Aer is not installed or importable.")

In [ ]:
import importlib.metadata
import sys

print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}") # Good to know which env you're in

gpu_installed = False
std_installed = False

try:
    gpu_version = importlib.metadata.version('qiskit-aer-gpu')
    print(f"[OK] qiskit-aer-gpu is installed, version: {gpu_version}")
    gpu_installed = True
except importlib.metadata.PackageNotFoundError:
    print("[INFO] qiskit-aer-gpu does NOT seem to be installed.")

try:
    std_version = importlib.metadata.version('qiskit-aer')
    print(f"[WARNING] Standard 'qiskit-aer' (version {std_version}) is also installed.")
    print("          This might cause issues. 'qiskit-aer-gpu' should replace 'qiskit-aer'.")
    print("          Consider uninstalling 'qiskit-aer': pip uninstall qiskit-aer")
    std_installed = True
except importlib.metadata.PackageNotFoundError:
    if gpu_installed:
         print("[OK] Standard 'qiskit-aer' is not installed (which is correct when using qiskit-aer-gpu).")
    # else: # Neither is installed - already covered by the first check failing

if not gpu_installed and not std_installed:
    print("[ERROR] Neither 'qiskit-aer-gpu' nor 'qiskit-aer' seem to be installed.")
elif not gpu_installed and std_installed:
    print("[INFO] Only standard 'qiskit-aer' is installed. You are likely using the CPU version.")

In [ ]:
# Imports
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# Load dataset (4-digit classification)
digits = load_digits()
X, y = digits.data, digits.target
mask = (y >= 0) & (y <= 3)
X, y = X[mask], y[mask]

# Standardize & PCA (match qubits)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
nqubits = 8
pca = PCA(n_components=nqubits)
X_reduced = pca.fit_transform(X_scaled)

# Split data
xs_tr, xs_test, y_tr, y_test = train_test_split(X_reduced, y, test_size=0.25, random_state=42)

# Feature map & GPU simulator
zzfm = ZZFeatureMap(feature_dimension=nqubits, reps=2, entanglement='full')
simulator_gpu = AerSimulator(device='GPU', method='statevector')

# Quantum Kernel directly with simulator GPU (no ComputeUncompute needed)
quantum_kernel_gpu = FidelityQuantumKernel(feature_map=zzfm, sampler=simulator_gpu)

# QSVC
qsvm_gpu = QSVC(quantum_kernel=quantum_kernel_gpu)

# Training
qsvm_gpu.fit(xs_tr, y_tr)

# Prediction & evaluation
y_pred_gpu = qsvm_gpu.predict(xs_test)
accuracy_gpu = accuracy_score(y_test, y_pred_gpu)

print(f"GPU-accelerated QSVC accuracy: {accuracy_gpu:.4f}")


In [ ]:
# GEMINI

In [ ]:
# Imports (use Aer's V2 Estimator)
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from qiskit.circuit.library import ZZFeatureMap
# Import the V2 Estimator FROM QISKIT AER primitives
from qiskit_aer.primitives import Estimator as AerEstimator # <--- Use Aer's V2 Estimator
# NO ComputeUncompute import needed
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# --- Load dataset and preprocess as before ---
digits = load_digits()
X, y = digits.data, digits.target
mask = (y >= 0) & (y <= 3)
X, y = X[mask], y[mask]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
nqubits = 8
pca = PCA(n_components=nqubits)
X_reduced = pca.fit_transform(X_scaled)

xs_tr, xs_test, y_tr, y_test = train_test_split(X_reduced, y, test_size=0.25, random_state=42)

# --- Feature map setup as before ---
zzfm = ZZFeatureMap(feature_dimension=nqubits, reps=2, entanglement='full')

# --- Configure Aer V2 Estimator ---

# 1. Instantiate Qiskit Aer's V2 Estimator.
#    Configure it for GPU statevector simulation using backend_options.
gpu_estimator_aer_v2 = AerEstimator(
    backend_options={
        "method": "statevector",
        "device": "GPU"
    },
    run_options={
        # Estimator calculates expectation values, shots=None is exact for statevector
        "shots": None,
        # Add other run options like seed if needed: "seed_simulator": 12345
    }
    # Add Estimator-specific options if needed (e.g., precision)
)

# --- Instantiate the Quantum Kernel ---
# TRY USING ESTIMATOR: Pass the configured Aer V2 Estimator.
# Use the 'estimator' keyword. Let the kernel use its default fidelity method.
quantum_kernel_gpu = FidelityQuantumKernel(
    feature_map=zzfm,
    estimator=gpu_estimator_aer_v2  # <--- Pass the estimator using 'estimator=' keyword
    # No 'sampler' argument provided
    # No 'fidelity' argument specified, let it use the default logic
)

# --- QSVC setup and execution remains the same ---
qsvm_gpu = QSVC(quantum_kernel=quantum_kernel_gpu)

# Training (Kernel evaluation should now use the provided estimator)
qsvm_gpu.fit(xs_tr, y_tr)

# Prediction & evaluation
y_pred_gpu = qsvm_gpu.predict(xs_test)
accuracy_gpu = accuracy_score(y_test, y_pred_gpu)

print(f"GPU-accelerated QSVC accuracy: {accuracy_gpu:.4f}")

In [ ]:
# NEW VERSION TO PREVENT KERNEL CRASH
import time
from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.metrics import accuracy_score

start_time = time.time()

# GPU-accelerated simulator (fully compatible)
simulator_gpu = AerSimulator(
    device='GPU',
    method='statevector',
    cuStateVec_enable=True
)

# ZZFeatureMap definition (10 qubits, stable reps=2)
# nqubits = 10
zzfm = ZZFeatureMap(feature_dimension=nqubits, reps=2, entanglement='full')

# FidelityQuantumKernel (no sampler needed explicitly)
quantum_kernel_gpu = FidelityQuantumKernel(feature_map=zzfm)

# QSVC with batch size explicitly reduced for stability
qsvm_gpu = QSVC(quantum_kernel=quantum_kernel_gpu)
qsvm_gpu.fit(xs_tr, y_tr)

# Evaluate and print results
y_pred_gpu = qsvm_gpu.predict(xs_test)
accuracy_gpu = accuracy_score(y_test, y_pred_gpu)
print(f"GPU-accelerated QSVC accuracy: {accuracy_gpu:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")





In [ ]:
time.time()